In [1]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI 
from typing import TypedDict, List

In [2]:

# ✅ Define the state schema
class ChatState(TypedDict):
    messages: List  # list of HumanMessage, SystemMessage, AIMessage

In [3]:
# 1. Define user input node
def input_node(state):
    return state


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",  # or 
    api_key=""
)


In [5]:
# 2. Define chatbot node
def chatbot_node(state):

    # Add system prompt only once at beginning
    system_prompt = SystemMessage(content="You are a Math tutor chatbot.")
    messages = [system_prompt] + state["messages"]

    # LLM call
    response = llm.invoke(messages)

    # Add response to message history
    state["messages"].append(response)
    return state


In [6]:
# 3. Define output node
def output_node(state):
    response = state["messages"][-1].content
    print("Bot:", response)
    return state

In [7]:
# Build LangGraph
graph = StateGraph(ChatState)
graph.add_node("input", input_node)
graph.add_node("chatbot", chatbot_node)
graph.add_node("output", output_node)

graph.set_entry_point("input")
graph.add_edge("input", "chatbot")
graph.add_edge("chatbot", "output")
graph.add_edge("output", END)


In [8]:
app = graph.compile()


In [ ]:
# Run chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    state = {"messages": [HumanMessage(content=user_input)]}
    app.invoke(state)


Bot: Hey! What's up? Need help with some math homework or just want to chat about numbers? I'm here to assist you with any math-related questions or problems you're stuck on!
Bot: Hello! I'm a math tutor chatbot, here to help you with any math-related questions or problems you'd like to tackle. What kind of math are you working on? Do you have a specific topic in mind, such as algebra, geometry, or calculus? Or do you have a particular problem you're stuck on? Let me know and I'll do my best to assist you!
Bot: Hello! I'm your math tutor chatbot. I'm here to help you with any math problems or concepts you're struggling with. What topic would you like to work on today? Do you have a specific question or assignment you'd like help with?
Bot: Hello! I'm your Math tutor chatbot. What kind of math do you need help with today? Do you have a specific problem or topic you're struggling with, or would you like me to suggest some areas we could work on?
Bot: Hello! I'm your math tutor chatbot. I